<a href="https://colab.research.google.com/github/digwit678/Can-Language-Models-Follow-Discussions/blob/main/Probing_Data_Post_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install igraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 35.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pickle
import igraph as ig
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
PATH_DATA_PROBING_1 = '/content/drive/MyDrive/Can Language Models Follow Discussions?/1/probing files v1/folds_1_v3.csv'

In [ ]:
import pandas as pd
df = pd.read_csv(PATH_DATA_PROBING_1)

In [ ]:
df

,index,inputs,context,topic,org_label,string_label,label,set-0
0,0,('För mycket jobb att tänka på detta som stör ...,NaN,Ska Arcada gå inför en aktivitetsbaserad arbet...,Con,0,0,train
1,1,"('YES! When people do something so horrific, t...",NaN,Should the death penalty be allowed?,Con,0,0,train
2,2,('We would get a better understanding of the c...,NaN,Should college cost be more regulated by the g...,Pro,1,1,train
3,3,('This would make it easier for fresh faces wi...,NaN,Congress should have limited terms.,Con,0,0,train
4,4,('Friendships with ex-partners are likely to b...,NaN,Should I stay friends with my ex-partner while...,Pro,1,1,train
...,...,...,...,...,...,...,...,...
7237,7237,('Maus es una ficción porque a pesar de estar ...,NaN,Maus es una no ficción porque trata acerca de ...,Pro,1,1,test
7238,7238,('Article 11\\(1\\) prohibits for-profit web s...,NaN,The EU was right to approve the [Directive on ...,Pro,1,1,test
7239,7239,('Modern industrial medicine has improved heal...,NaN,Has the Industrial Revolution and its conseque...,Pro,1,1,test
7240,7240,"('Emotions relate to ethical judgements.', 'Em...",NaN,Are our emotions a useful potential source of ...,Pro,1,1,test


In [ ]:
df.topic.head()

0    Ska Arcada gå inför en aktivitetsbaserad arbet...
1                 Should the death penalty be allowed?
2    Should college cost be more regulated by the g...
3                  Congress should have limited terms.
4    Should I stay friends with my ex-partner while...
Name: topic, dtype: object

# Data Cleaning pt 4: Removing Non-English Sentences

In [ ]:
!pip install langdetect
import langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=101b72acba127301d3e597761d0fec6bbfb78881c5b62c86e59b3a0e580fb6af
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
from langdetect import detect


In [ ]:
def clean_text(text):
    # Remove URLs and special characters to help detection
    return re.sub(r'http\S+|\\|[@#$%^&*()_+={}\[\]:;"\'<>,.?/`~\n]', '', text)


In [ ]:
def is_sentence_english(sentence):
    sentence_cleaned = clean_text(sentence)
    try:
        return detect(sentence_cleaned) == 'en'
    except:
        return False  # If language detection fails, assume non-English


In [ ]:
# Checking if the topic is written in English or not.
# We assume that English topics contain English claims and non-English topics contain non-English claims.
df['is_english_topic'] = df['topic'].apply(is_sentence_english)
df.head()


,index,inputs,context,topic,org_label,string_label,label,set-0,is_english_topic
0,0,('För mycket jobb att tänka på detta som stör ...,NaN,Ska Arcada gå inför en aktivitetsbaserad arbet...,Con,0,0,train,False
1,1,"('YES! When people do something so horrific, t...",NaN,Should the death penalty be allowed?,Con,0,0,train,True
2,2,('We would get a better understanding of the c...,NaN,Should college cost be more regulated by the g...,Pro,1,1,train,True
3,3,('This would make it easier for fresh faces wi...,NaN,Congress should have limited terms.,Con,0,0,train,True
4,4,('Friendships with ex-partners are likely to b...,NaN,Should I stay friends with my ex-partner while...,Pro,1,1,train,True


In [ ]:
# Filter DataFrame based on this new column
df_filtered_by_topic = df[df['is_english_topic']]

# Drop the 'is_english_topic' column if it's no longer needed
df_filtered_by_topic.drop('is_english_topic', axis=1, inplace=True)


   index                                             inputs  context  \
1      1  ('YES! When people do something so horrific, t...      NaN   
2      2  ('We would get a better understanding of the c...      NaN   
3      3  ('This would make it easier for fresh faces wi...      NaN   
4      4  ('Friendships with ex-partners are likely to b...      NaN   
6      6  ('Cats have lived [alongside](https://www.thea...      NaN   

                                               topic org_label  string_label  \
1               Should the death penalty be allowed?       Con             0   
2  Should college cost be more regulated by the g...       Pro             1   
3                Congress should have limited terms.       Con             0   
4  Should I stay friends with my ex-partner while...       Pro             1   
6                   Cats should not be kept as pets.       Con             0   

   label  set-0  
1      0  train  
2      1  train  
3      0  train  
4      1  trai

<ipython-input-42-40a80187dfc1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_by_topic.drop('is_english_topic', axis=1, inplace=True)


In [ ]:
df_filtered_by_topic.head()

,index,inputs,context,topic,org_label,string_label,label,set-0
1,1,"('YES! When people do something so horrific, t...",NaN,Should the death penalty be allowed?,Con,0,0,train
2,2,('We would get a better understanding of the c...,NaN,Should college cost be more regulated by the g...,Pro,1,1,train
3,3,('This would make it easier for fresh faces wi...,NaN,Congress should have limited terms.,Con,0,0,train
4,4,('Friendships with ex-partners are likely to b...,NaN,Should I stay friends with my ex-partner while...,Pro,1,1,train
6,6,('Cats have lived [alongside](https://www.thea...,NaN,Cats should not be kept as pets.,Con,0,0,train


In [ ]:
f"{len(df) - len(df_filtered_by_topic)} rows of non-English topics were removed"

'1727 rows of non-English topics were removed'

In [ ]:
df.to_csv('/content/drive/MyDrive/Can Language Models Follow Discussions?/1/folds_1_v3_only_english.csv', index=True)

# Manual Inspection of 30 Randomly Selected Example Rows

In [ ]:
import pandas as pd
import random

# Create a function to randomly select 30 rows from a DataFrame
def select_random_rows(df, num_rows=30):
    """
    Select a random subset of rows from a DataFrame.

    Parameters:
        df (DataFrame): The original DataFrame.
        num_rows (int): The number of rows to randomly select.

    Returns:
        DataFrame: A new DataFrame containing randomly selected rows.
    """
    # Check if the DataFrame has enough rows
    if len(df) < num_rows:
        print("The DataFrame has fewer rows than the number specified for selection.")
        return df
    else:
        # Randomly select 'num_rows' row indices
        random_indices = random.sample(list(df.index), num_rows)

        # Select rows with the random indices and return
        return df.loc[random_indices]

In [ ]:
# Randomly selecting 30 rows from the filtered DataFrame to manually verify for the 1st probing round
thirty_random_rows = select_random_rows(df_filtered_by_topic)
print(f"{len(thirty_random_rows)} rows randomly selected: \n")
thirty_random_rows

30 rows randomly selected: 



,index,inputs,context,topic,org_label,string_label,label,set-0
2570,2570,('Art is education and reflection based on inn...,NaN,Is it possible to define art?,Pro,1,1,train
2439,2439,('Children are too young to be able to provide...,NaN,First Aid should be taught in schools worldwide.,Pro,1,1,train
2892,2892,"(""Craster's sons are given to the woods."", 'Ar...",NaN,Just about every Game of Thrones baby was swit...,Pro,1,1,train
2255,2255,('Human rights are more important than state l...,NaN,The captain of Sea Watch 3 decided right by en...,Con,0,0,train
4866,4866,('This statistic does not appear to consider a...,NaN,One study found that [88%](https://fightthenew...,Pro,1,1,dev
4411,4411,('Reusing tires is an economical solution for ...,NaN,Using used tires is dangerous and should not h...,Pro,1,1,dev
1010,1010,('If electricity were to suddenly stop working...,NaN,Imagine that our power supply fails; a global ...,Pro,1,1,train
1658,1658,"('Dingoes are great hunters, big boss man stuf...",NaN,Dingoes should not be used as a form of biocon...,Pro,1,1,train
5734,5734,"('Well he might be a bit shy doe', 'Sick video')",NaN,Get memeulous to take his mask and hat off,Con,0,0,dev
3396,3396,"(""Artifice is only a mask, and can therefore b...",NaN,The most essential skill in political theater ...,Con,0,0,train


2892: acc to https://g.co/bard/share/b0f7bde8d81b pro is ok

1502: sound contradictory but maybe the user means the same: difficult for the model   
2372: all answers are pro (multiple choice),   (should be in train set)  
2341: spam ,   
6865: all answers are pro (multiple choice)  (should be in test set)   

so 1502 and 2341 are going to be replaced

In [ ]:
two_random_rows = select_random_rows(df_filtered_by_topic,2)

In [ ]:
two_random_rows

,index,inputs,context,topic,org_label,string_label,label,set-0
1299,1299,('There are people who do not conform to the g...,NaN,The gender binary corresponds to the sex binary.,Con,0,0,train
2015,2015,('Constant price changes regardless of inflati...,NaN,Competition is a driving force of the economy.,Pro,1,1,train


In [ ]:
def replace_rows(original_df, rows_to_remove, rows_to_add):
    df_removed = original_df.drop(rows_to_remove)
    df_replaced = pd.concat([df_removed, rows_to_add]).reset_index(drop=True)
    return df_replaced

# Replacing rows with indices 1502 and 2341
df_replaced = replace_rows(thirty_random_rows, [1502, 2341], two_random_rows)

In [ ]:
# Splitting the data again to maintain the train,validation,test ratio
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_replaced, test_size=0.2, random_state=42)
train, dev = train_test_split(train, test_size=0.25, random_state=42)

# Assign set labels
train['set-0'] = 'train'
dev['set-0'] = 'dev'
test['set-0'] = 'test'


In [ ]:
# ensuring the multiple-choice examples are not in the same set
#df_replaced.loc[2372, 'set-0'] = 'train'
#df_replaced.loc[6865, 'set-0'] = 'test'
#rows_to_remove = [2372, 6865]
#df_selected = df_replaced.drop(rows_to_remove)

In [ ]:
df_replaced

,index,inputs,context,topic,org_label,string_label,label,set-0
0,2570,('Art is education and reflection based on inn...,NaN,Is it possible to define art?,Pro,1,1,train
1,2439,('Children are too young to be able to provide...,NaN,First Aid should be taught in schools worldwide.,Pro,1,1,train
2,2892,"(""Craster's sons are given to the woods."", 'Ar...",NaN,Just about every Game of Thrones baby was swit...,Pro,1,1,train
3,2255,('Human rights are more important than state l...,NaN,The captain of Sea Watch 3 decided right by en...,Con,0,0,train
4,4866,('This statistic does not appear to consider a...,NaN,One study found that [88%](https://fightthenew...,Pro,1,1,dev
5,4411,('Reusing tires is an economical solution for ...,NaN,Using used tires is dangerous and should not h...,Pro,1,1,dev
6,1010,('If electricity were to suddenly stop working...,NaN,Imagine that our power supply fails; a global ...,Pro,1,1,train
7,1658,"('Dingoes are great hunters, big boss man stuf...",NaN,Dingoes should not be used as a form of biocon...,Pro,1,1,train
8,5734,"('Well he might be a bit shy doe', 'Sick video')",NaN,Get memeulous to take his mask and hat off,Con,0,0,dev
9,3396,"(""Artifice is only a mask, and can therefore b...",NaN,The most essential skill in political theater ...,Con,0,0,train


In [ ]:
df_selected.to_csv('/content/drive/MyDrive/Can Language Models Follow Discussions?/1/folds_1_thirty_samples.csv', index=True)

In [ ]:
import pandas as pd
import yaml
import os

def save_final_files_to_drive(probing_example_nr, df, num_inputs, subsample=False, additional_info="", control_task_type="NONE", num_labels=2, num_probe_folds=1, probe_task_type="", probes_samples_path="sample_probes"):
    """
    Save the final probing files to Google Drive.

    Parameters:
    - probing_example_nr: The probing example number used in the folder path.
    - df: The DataFrame to save.
    - num_inputs: Number of input sentences (1 for single sentence, 2 for two sentences).
    - subsample: Boolean flag to indicate if it's a subsample or full data (default: False).
    - additional_info:  is added to the end of the probe name if specified
    - control_task_type: The type of control task (default: "NONE").
    - num_labels: Number of labels (default: 2).
    - num_probe_folds: Number of probing folds (default: 1).
    - probe_task_type: The task type of the probe (default: "SENTENCE").
    - probes_samples_path: Relative path to the probe (default: "sample_probes").
    """

    # Determine folder based on subsample or full
    folder_type = "subsample" if subsample else "full"

    # Set the base path
    base_path = f"/content/drive/MyDrive/Can Language Models Follow Discussions?/{probing_example_nr}/final probing files/{folder_type}/"

    # Create folder if not exists
    if not os.path.exists(base_path):
        os.makedirs(base_path)

    # Determine the config file name
    config_name = "config"
    if num_inputs == 2:
        config_name += "-bi"
    if subsample:
        config_name += "-rand"
    config_name += "-none.yaml"

    # Create YAML config content
    config_content = {
        'control_task_type': control_task_type,
        'num_inputs': num_inputs,
        'num_labels': num_labels,
        'num_probe_folds': num_probe_folds,
        'probe_name': f"probe-{('single' if num_inputs == 1 else 'two')}_sentence_probing_task_{(additional_info if additional_info != '' else '')}_{probing_example_nr}",
        'probe_task_type': ("SENTENCE" if num_inputs == 1 else "SENTENCE_PAIR_BI") if probe_task_type =="" else probe_task_type,
        'probes_samples_path': f"{probes_samples_path}/{('single' if num_inputs == 1 else 'two')}_sentence_probing_task"
    }

    # Save YAML file
    with open(base_path + config_name, 'w') as file:
        yaml.safe_dump(config_content, file, default_flow_style=False)

    # Save DataFrame as CSV
    df.to_csv(base_path + "folds.csv", index=True)

    print(f"Files saved at {base_path}")


save_final_files_to_drive(1, df_selected, num_inputs=2, subsample=True)


Files saved at /content/drive/MyDrive/Can Language Models Follow Discussions?/1/final probing files/subsample/


In [ ]:
save_final_files_to_drive(1, df_filtered_by_topic, num_inputs=2, subsample=False)

Files saved at /content/drive/MyDrive/Can Language Models Follow Discussions?/1/final probing files/full/
